In [1]:
import warnings
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5f}'.format

In [2]:
import pandas as pd

df = pd.read_csv('./final/pre_melt.csv')
df

,자치구,상권_코드,서비스_업종_코드_명,총_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_상주인구_수,연령대_40_상주인구_수,연령대_50_상주인구_수,연령대_60_이상_상주인구_수,유사_업종_점포_수,총_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,집객시설수,교통시설수,당월_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,cluster,geometry,집객력(점수),매출액(점수),경쟁업체(점수),편의성(점수),잠재고객(점수)
0,관악구,1,PC방,4539089.50000,298619.25000,1434923.50000,955030.25000,690792.75000,493429.25000,666293.25000,9588,437,1037,3103,1809,1328,1874,7,1580,1,317,450,449,266,97,0.00000,0.00000,1133741818.50000,34712409.50000,626361694.75000,384977663.00000,46826285.00000,29762628.25000,11101138.00000,0,POLYGON ((126.93440736942736 37.46765303321722...,1,5,5,1,4.33333
1,서초구,2,PC방,16273249.00000,1207452.75000,4389792.25000,4411572.75000,3071175.25000,1708511.75000,1484744.50000,16458,1503,1557,4477,3368,2134,3419,16,144420,391,31353,46332,37292,21664,7388,658.00000,59.00000,1289435274.25000,71259296.00000,637215689.75000,330174688.50000,111611900.50000,102094911.50000,27678678.00000,0,POLYGON ((127.02248470599385 37.49640036649937...,5,5,5,5,5.00000
2,광진구,3,PC방,2924851.00000,312811.25000,1203307.75000,479611.00000,325448.00000,275166.50000,328506.00000,4782,209,476,1461,645,500,1491,6,892,5,148,146,178,204,211,83.00000,9.00000,401440972.25000,31942312.50000,253757717.50000,66830561.50000,16971251.50000,29619195.25000,2319934.00000,0,POLYGON ((127.06601812263857 37.54077443531351...,5,4,4,4,3.66667
3,노원구,4,PC방,4589353.75000,760577.50000,881616.75000,663095.00000,663288.50000,716511.50000,904265.25000,899,26,101,212,146,163,251,10,1552,9,369,295,343,307,229,190.00000,28.00000,812963814.75000,39189698.25000,532913535.25000,131774239.25000,34662812.25000,61803012.25000,12620517.50000,0,POLYGON ((127.06000553082049 37.65328168143866...,5,5,5,5,3.00000
4,서초구,5,PC방,8102497.25000,564253.00000,1787831.00000,2028383.75000,1639661.25000,1014404.75000,1067962.75000,12777,762,932,3174,2854,1953,3102,10,37704,103,8177,10755,8587,5610,4472,322.00000,27.00000,2865765169.50000,125861489.75000,1473505181.50000,801599281.25000,187295560.25000,214415608.25000,59295092.00000,0,POLYGON ((127.02213215191853 37.51412093435149...,5,5,5,5,5.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17409,강남구,17410,화초,1146115.50000,525059.50000,106693.00000,86723.75000,179014.50000,131369.50000,117255.00000,1319,113,302,145,147,379,233,6,17175,202,3397,3819,3179,4061,2517,0.00000,0.00000,29851309.00000,654936.50000,3033389.50000,3658779.75000,3949314.75000,5515253.00000,2378453.00000,3,POLYGON ((127.05624428377789 37.49739863928150...,1,2,3,1,3.33333
17410,중구,17411,화초,856378.50000,28532.75000,162665.50000,241989.00000,212396.00000,126906.50000,83889.00000,73,7,3,2,16,7,38,6,30610,14,4275,11589,8473,5661,598,0.00000,0.00000,242797052.50000,0.00000,6078498.75000,19118030.00000,17026629.25000,5586503.25000,2266109.25000,3,POLYGON ((126.98279757109825 37.56903233158031...,1,5,3,1,2.66667
17411,종로구,17412,화초,416415.75000,28962.75000,64605.75000,99649.00000,99717.25000,68454.50000,55027.50000,279,16,16,45,41,50,111,9,12937,9,1442,4392,4485,2080,529,0.00000,0.00000,158127575.00000,0.00000,4344466.00000,20422872.00000,34284314.75000,47085967.25000,15004763.75000,3,POLYGON ((126.98050412101318 37.57615189223551...,1,4,4,1,2.33333
17412,중구,17413,화초,453583.25000,33486.25000,63577.75000,103645.00000,101260.50000,74842.00000,76772.25000,1598,123,104,273,301,269,528,9,2190,2,393,736,598,295,166,0.00000,0.00000,779603873.25000,0.00000,11099656.25000,13809593.50000,24453636.50000,29322696.75000,46248438.00000,3,"POLYGON ((126.9660870334594 37.56157461963889,...",1,5,4,1,2.00000


In [3]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area if geom_str is not None else None)

In [4]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area_m2'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area * (111.32**2) * (10**6) if geom_str is not None else None)

In [5]:
from shapely.wkt import loads

# Assuming df['geometry'] contains a WKT-formatted POLYGON string
df['area_km2'] = df['geometry'].apply(lambda geom_str: loads(geom_str).area * (111.32**2) if geom_str is not None else None)

In [6]:
df_l = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', 'cluster']]
df_r = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수', '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수', 'cluster']]
df_w = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수', 'cluster']]
df_f = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '집객시설수', '교통시설수', 'cluster']]
df_s = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '유사_업종_점포_수', 'cluster']]
df_e = df[['자치구', '상권_코드', '서비스_업종_코드_명', 'area_m2', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', 'cluster']]

In [7]:
df_l = df_l.rename(columns = {'연령대_10_유동인구_수' : '10대',
                                             '연령대_20_유동인구_수' : '20대',
                                             '연령대_30_유동인구_수' : '30대',
                                             '연령대_40_유동인구_수' : '40대',
                                             '연령대_50_유동인구_수' : '50대',
                                             '연령대_60_이상_유동인구_수' : '60대 이상'})

In [8]:
df_l = pd.melt(df_l, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                      var_name = '연령', value_name = '유동인구수')

In [9]:
df_r = df_r.rename(columns = {'연령대_10_상주인구_수' : '10대',
                                             '연령대_20_상주인구_수' : '20대',
                                             '연령대_30_상주인구_수' : '30대',
                                             '연령대_40_상주인구_수' : '40대',
                                             '연령대_50_상주인구_수' : '50대',
                                             '연령대_60_이상_상주인구_수' : '60대 이상'})

In [10]:
df_r = pd.melt(df_r, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                      var_name = '연령', value_name = '상주인구수')

In [11]:
df_w = df_w.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                                             '연령대_20_직장_인구_수' : '20대',
                                             '연령대_30_직장_인구_수' : '30대',
                                             '연령대_40_직장_인구_수' : '40대',
                                             '연령대_50_직장_인구_수' : '50대',
                                             '연령대_60_이상_직장_인구_수' : '60대 이상'})

In [12]:
df_w = pd.melt(df_w, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                      var_name = '연령', value_name = '직장인구수')

In [13]:
df_e = df_e.rename(columns = {'연령대_10_매출_금액' : '10대',
                                             '연령대_20_매출_금액' : '20대',
                                             '연령대_30_매출_금액' : '30대',
                                             '연령대_40_매출_금액' : '40대',
                                             '연령대_50_매출_금액' : '50대',
                                             '연령대_60_이상_매출_금액' : '60대 이상'})

In [14]:
df_e = pd.melt(df_e, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster', 'area_m2'],
                      var_name = '연령', value_name = '매출금액')

In [15]:
melt_df = pd.merge(df_l, df_r)
melt_df = pd.merge(melt_df, df_w)
melt_df = pd.merge(melt_df, df_f)
melt_df = pd.merge(melt_df, df_s)
melt_df = pd.merge(melt_df, df_e)

In [16]:
melt_df

,자치구,상권_코드,서비스_업종_코드_명,cluster,area_m2,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액
0,관악구,1,PC방,0,317032.35415,10대,298619.25000,437,1,0.00000,0.00000,7,34712409.50000
1,관악구,1,PC방,0,317032.35415,20대,1434923.50000,1037,317,0.00000,0.00000,7,626361694.75000
2,관악구,1,PC방,0,317032.35415,30대,955030.25000,3103,450,0.00000,0.00000,7,384977663.00000
3,관악구,1,PC방,0,317032.35415,40대,690792.75000,1809,449,0.00000,0.00000,7,46826285.00000
4,관악구,1,PC방,0,317032.35415,50대,493429.25000,1328,266,0.00000,0.00000,7,29762628.25000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104479,서초구,17414,화초,3,473340.16191,20대,293923.00000,537,1553,0.00000,0.00000,12,621016.75000
104480,서초구,17414,화초,3,473340.16191,30대,449062.25000,1111,3044,0.00000,0.00000,12,2797530.25000
104481,서초구,17414,화초,3,473340.16191,40대,390172.75000,1087,2666,0.00000,0.00000,12,4205668.00000
104482,서초구,17414,화초,3,473340.16191,50대,314164.75000,981,1705,0.00000,0.00000,12,1718575.75000


In [17]:
original_col = ['유동인구수', '상주인구수', '직장인구수', '집객시설수', '교통시설수', '유사_업종_점포_수', '매출금액']
area_col = ['단위_유동인구수', '단위_상주인구수', '단위_직장인구수', '단위_집객시설수', '단위_교통시설수', '단위_유사_업종_점포_수', '단위_매출금액']

for i, j in zip(original_col, area_col) :
    for k in range(len(melt_df)):
        if melt_df.loc[k,i] == 0:
            melt_df.loc[k,j] = melt_df.loc[k,i]
        else :
            melt_df.loc[k,j] = melt_df.loc[k,i] / melt_df.loc[k,'area_m2']

In [18]:
melt_df

,자치구,상권_코드,서비스_업종_코드_명,cluster,area_m2,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액,단위_유동인구수,단위_상주인구수,단위_직장인구수,단위_집객시설수,단위_교통시설수,단위_유사_업종_점포_수,단위_매출금액
0,관악구,1,PC방,0,317032.35415,10대,298619.25000,437,1,0.00000,0.00000,7,34712409.50000,0.94192,0.00138,0.00000,0.00000,0.00000,0.00002,109.49169
1,관악구,1,PC방,0,317032.35415,20대,1434923.50000,1037,317,0.00000,0.00000,7,626361694.75000,4.52611,0.00327,0.00100,0.00000,0.00000,0.00002,1975.70275
2,관악구,1,PC방,0,317032.35415,30대,955030.25000,3103,450,0.00000,0.00000,7,384977663.00000,3.01241,0.00979,0.00142,0.00000,0.00000,0.00002,1214.31664
3,관악구,1,PC방,0,317032.35415,40대,690792.75000,1809,449,0.00000,0.00000,7,46826285.00000,2.17893,0.00571,0.00142,0.00000,0.00000,0.00002,147.70191
4,관악구,1,PC방,0,317032.35415,50대,493429.25000,1328,266,0.00000,0.00000,7,29762628.25000,1.55640,0.00419,0.00084,0.00000,0.00000,0.00002,93.87884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104479,서초구,17414,화초,3,473340.16191,20대,293923.00000,537,1553,0.00000,0.00000,12,621016.75000,0.62096,0.00113,0.00328,0.00000,0.00000,0.00003,1.31199
104480,서초구,17414,화초,3,473340.16191,30대,449062.25000,1111,3044,0.00000,0.00000,12,2797530.25000,0.94871,0.00235,0.00643,0.00000,0.00000,0.00003,5.91019
104481,서초구,17414,화초,3,473340.16191,40대,390172.75000,1087,2666,0.00000,0.00000,12,4205668.00000,0.82430,0.00230,0.00563,0.00000,0.00000,0.00003,8.88509
104482,서초구,17414,화초,3,473340.16191,50대,314164.75000,981,1705,0.00000,0.00000,12,1718575.75000,0.66372,0.00207,0.00360,0.00000,0.00000,0.00003,3.63074


In [19]:
from sklearn.preprocessing import MinMaxScaler

trans_col = ['정규_유동인구수', '정규_상주인구수', '정규_직장인구수', '정규_집객시설수', '정규_교통시설수', '정규_유사_업종_점포_수', '정규_매출금액']

scaler = MinMaxScaler()
melt_df[trans_col] = scaler.fit_transform(melt_df[area_col])

In [20]:
melt_df

,자치구,상권_코드,서비스_업종_코드_명,cluster,area_m2,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액,단위_유동인구수,단위_상주인구수,단위_직장인구수,단위_집객시설수,단위_교통시설수,단위_유사_업종_점포_수,단위_매출금액,정규_유동인구수,정규_상주인구수,정규_직장인구수,정규_집객시설수,정규_교통시설수,정규_유사_업종_점포_수,정규_매출금액
0,관악구,1,PC방,0,317032.35415,10대,298619.25000,437,1,0.00000,0.00000,7,34712409.50000,0.94192,0.00138,0.00000,0.00000,0.00000,0.00002,109.49169,0.04739,0.06014,0.00003,0.00000,0.00000,0.00035,0.00006
1,관악구,1,PC방,0,317032.35415,20대,1434923.50000,1037,317,0.00000,0.00000,7,626361694.75000,4.52611,0.00327,0.00100,0.00000,0.00000,0.00002,1975.70275,0.22771,0.14271,0.00891,0.00000,0.00000,0.00035,0.00103
2,관악구,1,PC방,0,317032.35415,30대,955030.25000,3103,450,0.00000,0.00000,7,384977663.00000,3.01241,0.00979,0.00142,0.00000,0.00000,0.00002,1214.31664,0.15156,0.42702,0.01265,0.00000,0.00000,0.00035,0.00063
3,관악구,1,PC방,0,317032.35415,40대,690792.75000,1809,449,0.00000,0.00000,7,46826285.00000,2.17893,0.00571,0.00142,0.00000,0.00000,0.00002,147.70191,0.10962,0.24895,0.01262,0.00000,0.00000,0.00035,0.00008
4,관악구,1,PC방,0,317032.35415,50대,493429.25000,1328,266,0.00000,0.00000,7,29762628.25000,1.55640,0.00419,0.00084,0.00000,0.00000,0.00002,93.87884,0.07830,0.18275,0.00748,0.00000,0.00000,0.00035,0.00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104479,서초구,17414,화초,3,473340.16191,20대,293923.00000,537,1553,0.00000,0.00000,12,621016.75000,0.62096,0.00113,0.00328,0.00000,0.00000,0.00003,1.31199,0.03124,0.04950,0.02924,0.00000,0.00000,0.00040,0.00000
104480,서초구,17414,화초,3,473340.16191,30대,449062.25000,1111,3044,0.00000,0.00000,12,2797530.25000,0.94871,0.00235,0.00643,0.00000,0.00000,0.00003,5.91019,0.04773,0.10240,0.05731,0.00000,0.00000,0.00040,0.00000
104481,서초구,17414,화초,3,473340.16191,40대,390172.75000,1087,2666,0.00000,0.00000,12,4205668.00000,0.82430,0.00230,0.00563,0.00000,0.00000,0.00003,8.88509,0.04147,0.10019,0.05019,0.00000,0.00000,0.00040,0.00000
104482,서초구,17414,화초,3,473340.16191,50대,314164.75000,981,1705,0.00000,0.00000,12,1718575.75000,0.66372,0.00207,0.00360,0.00000,0.00000,0.00003,3.63074,0.03339,0.09042,0.03210,0.00000,0.00000,0.00040,0.00000


In [22]:
melt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104484 entries, 0 to 104483
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   자치구            104484 non-null  object 
 1   상권_코드          104484 non-null  int64  
 2   서비스_업종_코드_명    104484 non-null  object 
 3   cluster        104484 non-null  int64  
 4   area_m2        104484 non-null  float64
 5   연령             104484 non-null  object 
 6   유동인구수          104484 non-null  float64
 7   상주인구수          104484 non-null  int64  
 8   직장인구수          104484 non-null  int64  
 9   집객시설수          104484 non-null  float64
 10  교통시설수          104484 non-null  float64
 11  유사_업종_점포_수     104484 non-null  int64  
 12  매출금액           104484 non-null  float64
 13  단위_유동인구수       104484 non-null  float64
 14  단위_상주인구수       104484 non-null  float64
 15  단위_직장인구수       104484 non-null  float64
 16  단위_집객시설수       104484 non-null  float64
 17  단위_교통시설수       104484 non-nul

In [21]:
melt_df.to_csv("./final/melt_df_1.csv", index = False)

In [15]:
import pandas as pd

df = pd.read_csv('./final/pre_melt_not.csv')
df

,자치구_코드_명,상권_코드_명,서비스_업종_코드_명,총_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_상주인구_수,연령대_40_상주인구_수,연령대_50_상주인구_수,연령대_60_이상_상주인구_수,유사_업종_점포_수,총_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,집객시설수,교통시설수,당월_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,cluster,geometry,집객력(점수),매출액(점수),경쟁업체(점수),편의성(점수),잠재고객(점수),geometry2
0,마포구,KB국민은행 망원동지점,PC방,836247.50,101555.75,130083.25,150746.75,147307.50,127301.00,179254.00,2759,327,302,420,454,446,810,3,284,0,13,47,93,76,55,0.0,0.0,1.645965e+08,10821044.50,6.633817e+07,4.729981e+07,24106779.00,13321027.25,2709653.75,2,POLYGON ((126.90628774141749 37.56017997759881...,1,2,2,1,2.000000,LINESTRING (126.90628774141749 37.560179977598...
1,강동구,강동역 4번(강풀만화거리),PC방,3260405.00,401393.25,444916.00,584695.25,530117.25,509957.00,789327.75,11657,1167,1062,2035,2035,1707,3651,4,774,3,87,144,177,164,199,25.0,7.0,3.698310e+08,6259805.00,1.954315e+08,1.138075e+08,21940660.50,28416859.50,3974618.50,3,POLYGON ((127.12905372083152 37.53691281314833...,3,4,3,3,4.000000,LINESTRING (127.12905372083152 37.536912813148...
2,송파구,거여역 1번(거여공원),PC방,2030521.75,309286.75,243323.50,297709.25,321688.50,351266.25,507247.75,5600,584,703,702,842,1028,1741,2,872,3,147,186,251,200,85,0.0,0.0,1.862056e+08,16149052.00,6.709229e+07,7.262411e+07,10994779.00,16619959.50,2725451.00,3,POLYGON ((127.14497236761629 37.49908204355591...,1,2,1,1,3.666667,LINESTRING (127.14497236761629 37.499082043555...
3,동대문구,경희대후문,PC방,1651082.25,231522.75,895128.75,144123.25,106483.00,106069.25,167755.00,2343,139,551,826,182,217,428,2,86,0,18,17,24,17,10,0.0,0.0,8.773463e+07,7200617.00,6.183157e+07,4.184979e+06,3874223.00,9206784.00,1436460.00,2,"POLYGON ((127.0555268718518 37.59987102869982,...",1,1,1,1,2.000000,LINESTRING (127.0555268718518 37.5998710286998...
4,관악구,관악구신사동주민센터,PC방,2014735.00,197005.00,484959.50,401566.75,283582.00,265130.50,382490.50,7384,629,835,1600,1213,1263,1844,3,1168,1,215,376,261,204,111,0.0,0.0,5.993089e+08,41107010.50,3.420282e+08,1.423146e+08,38753360.50,29992744.50,5112953.25,3,"POLYGON ((126.92146157164424 37.4840285154918,...",1,5,2,1,4.000000,LINESTRING (126.92146157164424 37.484028515491...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21671,용산구,용산용문시장,화초,633523.75,63569.25,86254.25,123368.25,98877.00,103855.50,157600.00,679,50,60,125,84,109,251,3,65,0,7,15,18,10,15,0.0,0.0,5.169155e+07,54371.50,1.393154e+06,3.192814e+06,3043106.25,8862588.75,17281044.75,0,POLYGON ((126.96010422854445 37.53787373520765...,1,3,1,1,1.333333,LINESTRING (126.96010422854445 37.537873735207...
21672,서대문구,"이화여대 3,5,7길 상점가",화초,961836.75,88295.50,455521.75,150204.00,95197.00,79641.50,92977.50,501,13,129,166,46,45,102,6,147,3,34,15,41,31,23,0.0,0.0,2.550474e+07,707053.50,1.062218e+07,2.821545e+06,2462143.75,4126153.00,1316844.00,0,POLYGON ((126.94638826105991 37.55915144165164...,1,2,3,1,1.333333,LINESTRING (126.94638826105991 37.559151441651...
21673,도봉구,창동신창시장,화초,847630.50,139434.25,104942.00,102118.25,128913.25,151671.25,220551.25,2288,229,296,302,347,423,691,4,116,0,5,24,35,31,21,0.0,0.0,4.573667e+06,62138.00,4.142525e+04,1.829082e+05,2918887.50,571668.50,640498.50,0,"POLYGON ((127.037012637493 37.64039507748914, ...",1,1,2,1,2.000000,LINESTRING (127.037012637493 37.64039507748914...
21674,성동구,행당시장상점가,화초,810677.00,87438.25,193967.00,145498.75,115004.25,124408.00,144360.50,1686,105,202,347,221,271,540,3,360,0,42,76,102,95,45,0.0,0.0,4.246007e+06,66376.25,4.438912e+05,1.387678e+06,62227.75,2109521.00,0.00,0,POLYGON ((127.03542265294423 37.56111098075576...,1,1,1,1,1.666667,LINESTRING (127.03542265294423 37.561110980755...


In [16]:
df_l = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', 'cluster']]
df_r = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수', '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수', 'cluster']]
df_w = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수', 'cluster']]
df_f = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '집객시설수', '교통시설수', 'cluster']]
df_s = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '유사_업종_점포_수', 'cluster']]
df_e = df[['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', 'cluster']]

In [17]:
df_l = df_l.rename(columns = {'연령대_10_유동인구_수' : '10대',
                                             '연령대_20_유동인구_수' : '20대',
                                             '연령대_30_유동인구_수' : '30대',
                                             '연령대_40_유동인구_수' : '40대',
                                             '연령대_50_유동인구_수' : '50대',
                                             '연령대_60_이상_유동인구_수' : '60대 이상'})

In [18]:
df_l = pd.melt(df_l, id_vars = ['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '유동인구수')

In [19]:
df_r = df_r.rename(columns = {'연령대_10_상주인구_수' : '10대',
                                             '연령대_20_상주인구_수' : '20대',
                                             '연령대_30_상주인구_수' : '30대',
                                             '연령대_40_상주인구_수' : '40대',
                                             '연령대_50_상주인구_수' : '50대',
                                             '연령대_60_이상_상주인구_수' : '60대 이상'})

In [20]:
df_r = pd.melt(df_r, id_vars = ['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '상주인구수')

In [21]:
df_w = df_w.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                                             '연령대_20_직장_인구_수' : '20대',
                                             '연령대_30_직장_인구_수' : '30대',
                                             '연령대_40_직장_인구_수' : '40대',
                                             '연령대_50_직장_인구_수' : '50대',
                                             '연령대_60_이상_직장_인구_수' : '60대 이상'})

In [22]:
df_w = pd.melt(df_w, id_vars = ['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '직장인구수')

In [23]:
df_e = df_e.rename(columns = {'연령대_10_매출_금액' : '10대',
                                             '연령대_20_매출_금액' : '20대',
                                             '연령대_30_매출_금액' : '30대',
                                             '연령대_40_매출_금액' : '40대',
                                             '연령대_50_매출_금액' : '50대',
                                             '연령대_60_이상_매출_금액' : '60대 이상'})

In [24]:
df_e = pd.melt(df_e, id_vars = ['자치구_코드_명', '상권_코드_명', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '매출금액')

In [25]:
melt_df_not = pd.merge(df_l, df_r)
melt_df_not = pd.merge(melt_df_not, df_w)
melt_df_not = pd.merge(melt_df_not, df_f)
melt_df_not = pd.merge(melt_df_not, df_s)
melt_df_not = pd.merge(melt_df_not, df_e)

In [29]:
melt_df_not

,자치구_코드_명,상권_코드_명,서비스_업종_코드_명,cluster,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액
0,마포구,KB국민은행 망원동지점,PC방,2,10대,101555.75,327,0,0.0,0.0,3,10821044.50
1,마포구,KB국민은행 망원동지점,PC방,2,20대,130083.25,302,13,0.0,0.0,3,66338174.00
2,마포구,KB국민은행 망원동지점,PC방,2,30대,150746.75,420,47,0.0,0.0,3,47299813.00
3,마포구,KB국민은행 망원동지점,PC방,2,40대,147307.50,454,93,0.0,0.0,3,24106779.00
4,마포구,KB국민은행 망원동지점,PC방,2,50대,127301.00,446,76,0.0,0.0,3,13321027.25
...,...,...,...,...,...,...,...,...,...,...,...,...
130051,마포구,홍대 걷고싶은 거리,화초,0,20대,257332.75,87,65,0.0,0.0,2,3765628.50
130052,마포구,홍대 걷고싶은 거리,화초,0,30대,126116.00,211,65,0.0,0.0,2,2621168.00
130053,마포구,홍대 걷고싶은 거리,화초,0,40대,74579.50,88,81,0.0,0.0,2,1509148.50
130054,마포구,홍대 걷고싶은 거리,화초,0,50대,48174.25,79,82,0.0,0.0,2,2091372.50


In [30]:
melt_df_not.to_csv("./final/melt_df_not.csv", index = False)